# Norms, conditions numbers and Eigensystems

In linear-algebra calculations, we are sometimes very unfortunate and have to solve a problem like $Ax = b$ (given fixed $A$), where small changes in $b$ produce extremely large changes in $x$. Such problems are said to be **ill-conditioned**.

This notebook explores this phenomenon. Along the way we will have to calculate condition numbers and eigenvalues.

## Preliminaries

Let's load numpy as usual, and the linear algebra package from numpy as we will find some functions in it useful. We also use the `GaussianElimination()` from one of the other notebooks and define the $L_2$-norm

In [1]:
import numpy as np
from numpy import linalg as la

In [2]:
def GaussianElimination(A, b):
    n = A.shape[1]
    
    # Append the vector b as a column to the matrix A
    A1 = np.c_[A,b]
    
    i = 0
    while i < n - 1:
        j = i+1
        while j < n:
            A1[j, i+1:] = A1[j, i+1:] - A1[i, i+1:]*A1[j,i]/A1[i,i]
            j += 1
        i += 1
        
    x = np.zeros(n)
    
    i = n-1
    while i >= 0:
        j = i
        x[i] = A1[i,n]
        while j < n-1:
            x[i] -= A1[i,j+1]*x[j+1]
            j += 1
        x[i] = x[i]/A1[i,i]
        i -= 1
    
    return x

In [3]:
def MatrixInverseViaGaussianElimination(A):
    n = A.shape[1]
    
    A1 = np.hstack((A,np.identity(n)))
    
    i = 0
    while i < n:
        j = 0
        while j < n:
            if(j == i): 
                j += 1
                continue
            A1[j] = (A1[j] - A1[i]*A1[j,i]/A1[i,i])
            A1[j] = A1[j]/A1[j,j]
            j += 1
        i += 1
    
    return A1[:,n:2*n]

In [4]:
def L2Norm(v):
    return la.norm(v,2)

# Norms

The $L^2$-norm of a matrix $A$ is formally defined by

$$\lVert A \rVert_2 = \sup_{x\neq0}\frac{\lVert Ax \rVert_2}{\lVert x \rVert} $$

For practical calculation, this is not a particularly useful definition. We derived a more useful formula:

$$ \lVert A \rVert_2 = \sqrt{\lambda_\text{max}} $$

where $\lambda_\text{max}$ is the maximum eigenvector of $A A^T$.

Let's check that NumPy's definitions agree with these formula.

In [31]:
A = np.random.rand(6,6) 

lmax = np.max(la.eig(A@A.T)[0])

la.norm(A,2) - np.sqrt(lmax)

-2.220446049250313e-15

Note that by default NumPy's `la.norm()` function computes the *Frobenius* norm for matrices. If you want the $L^2$-norm you need to call it as `la.norm(A,2)`, as above.

## Ill-condition system - example case

Let's look at an example where the matrix $A$ is ill-conditioned

In [6]:
A = np.array([[1.002,1],[1,0.998]])
b = np.array([2.002,1.998])

x = GaussianElimination(A,b)
print(x)

[1. 1.]


Slightly perturbing $b$ causes a huge change in the value of $x$

In [7]:
bp = np.array([2.0021,1.998])

xp = GaussianElimination(A,bp)
print(xp)

[-23.95  26.  ]


In [8]:
print("Change in b = %.4f%%" % (100*L2Norm(bp-b)/L2Norm(b)))
print("Change in x = %.2f%%" % (100*L2Norm(xp-x)/L2Norm(x)))

Change in b = 0.0035%
Change in x = 2497.50%


## Condition number

The condition number of a matrix $A$ is defined as 

$$ \kappa(A) = \lVert A \rVert_2 \lVert A^{-1} \rVert_2$$

We learned in the lectures that another practical way to compute this is via

$$\kappa(A) = \sqrt{\frac{\lambda_\text{max}}{\lambda_\text{min}}} $$

where $\lambda_\text{min/max}$ is the maximum eigenvalue of $A A^T$.

Let's use these two methods along with NumPy's built in method `la.cond()`

In [32]:
A = np.random.rand(6,6)

In [33]:
eigenvalues = la.eig(A.T@A)[0]
lmax = np.max(eigenvalues)
lmin = np.min(eigenvalues)

In [34]:
print("Condition number computed via norm definition:\t", la.norm(A,2)*la.norm(la.inv(A),2))
print("Condition number comuted via eigenvalue formula:", np.sqrt(lmax/lmin))
print("Condition number comuted via la.cond(A,2):\t", la.cond(A,2))

Condition number computed via norm definition:	 32.67118439809067
Condition number comuted via eigenvalue formula: 32.67118439809083
Condition number comuted via la.cond(A,2):	 32.67118439809068
